In [1]:
# 引入库包
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv('data/train.csv',header=None)
df_train.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age',"SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
df_train.drop(index=0,inplace=True)
df_train.head(5)
#PassengerId:乘客编号
#Survived：存活情况（存活：1，死亡：0）
#Pclass：客舱等级
#Name：乘客姓名
#Sex：性别
#Age：年龄
#SibSp：同乘的兄弟姐妹/配偶数
#Parch：同乘的父母/小孩数
#Ticket：船票编号
#Fare：船票价格
#Cabin：客舱号
#Embarked：登船港口

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NaN,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,NaN,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,NaN,S


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 12 columns):
PassengerId    891 non-null object
Survived       891 non-null object
Pclass         891 non-null object
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null object
SibSp          891 non-null object
Parch          891 non-null object
Ticket         891 non-null object
Fare           891 non-null object
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: object(12)
memory usage: 90.5+ KB


In [4]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
PassengerId,891,891,337,1,0.000000
Survived,891,2,0,549,0.000000
Pclass,891,3,3,491,0.000000
Name,891,891,"Widegren, Mr. Carl/Charles Peter",1,0.000000
Sex,891,2,male,577,0.000000
Age,714,88,24,30,0.198653
SibSp,891,7,0,608,0.000000
Parch,891,7,0,678,0.000000
Ticket,891,681,347082,7,0.000000
Fare,891,248,8.05,43,0.000000


In [5]:
#姓名特征处理
df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

title_Dict = {}
title_Dict.update(dict.fromkeys(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer'))
title_Dict.update(dict.fromkeys(['Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty'))
title_Dict.update(dict.fromkeys(['Mme', 'Ms', 'Mrs'], 'Mrs'))
title_Dict.update(dict.fromkeys(['Mlle', 'Miss'], 'Miss'))
title_Dict.update(dict.fromkeys(['Mr'], 'Mr'))
title_Dict.update(dict.fromkeys(['Master','Jonkheer'], 'Master'))

df_train['Title'] = df_train['Title'].map(title_Dict)

通过对Ticket数据的分析，我们可以看到部分票号数据有重复，同时结合亲属人数及名字的数据，和票价船舱等级对比，我们可以知道购买的票中有家庭票和团体票，所以我们需要将团体票的票价分配到每个人的头上

In [6]:
#船票编号特征处理
df_train['Fare'] = df_train['Fare'].astype("float")
df_train['Group_Ticket'] = df_train['Fare'].groupby(by=df_train['Ticket']).transform('count')
df_train['Fare'] = df_train['Fare'] / df_train['Group_Ticket']
df_train.drop(['Group_Ticket'], axis=1, inplace=True)

In [7]:
df_train.drop("PassengerId",1,inplace=True)
df_train.drop("Name",1,inplace=True)
df_train.drop("Ticket",1,inplace=True)

In [8]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
1,0,3,male,22,1,0,7.2500,NaN,S,Mr
2,1,1,female,38,1,0,71.2833,C85,C,Mrs
3,1,3,female,26,0,0,7.9250,NaN,S,Miss
4,1,1,female,35,1,0,26.5500,C123,S,Mrs
5,0,3,male,35,0,0,8.0500,NaN,S,Mr


In [9]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
Survived,891,2,0,549,0.000000
Pclass,891,3,3,491,0.000000
Sex,891,2,male,577,0.000000
Age,714,88,24,30,0.198653
SibSp,891,7,0,608,0.000000
Parch,891,7,0,678,0.000000
Cabin,204,147,G6,4,0.771044
Embarked,889,3,S,644,0.002245
Title,890,6,Mr,517,0.001122


In [10]:
df_train.loc[df_train.Cabin.isnull(), 'Cabin'] = 'U0'
df_train['Cabin'] = df_train['Cabin'].apply(lambda x: 0 if x == 'U0' else 1)

In [11]:
df_train.Embarked.isnull().value_counts()

False    889
True       2
Name: Embarked, dtype: int64

In [12]:
#删除数据集中某行的特征
def drop_feature_null_rows(df,feature):
    df[feature] = df[feature].fillna("999")#登船港口
    null_rows_index = df[(df[feature] == "999")].index.tolist()
    return df.drop(null_rows_index)

#删除登船港口的空行
df_train =drop_feature_null_rows(df_train,'Embarked')
df_train.shape

(889, 10)

In [13]:
df_train.select_dtypes(include="object").describe().T.assign(
  missing_pct=df_train.apply(lambda x : (len(x)-x.count())/float(len(x))))

,count,unique,top,freq,missing_pct
Survived,889,2,0,549,0.000000
Pclass,889,3,3,491,0.000000
Sex,889,2,male,577,0.000000
Age,712,88,24,30,0.199100
SibSp,889,7,0,606,0.000000
Parch,889,7,0,676,0.000000
Embarked,889,3,S,644,0.000000
Title,888,6,Mr,517,0.001125


In [14]:
df_train = pd.get_dummies(df_train,columns = ["Sex","Pclass","Embarked","Title",'Cabin'])
df_train.head()

,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,...,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Cabin_0,Cabin_1
1,0,22,1,0,7.2500,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0
2,1,38,1,0,71.2833,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
3,1,26,0,0,7.9250,1,0,0,0,1,...,0,1,0,1,0,0,0,0,1,0
4,1,35,1,0,26.5500,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
5,0,35,0,0,8.0500,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0


In [15]:
df_train.columns

Index(['Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'Title_Officer', 'Title_Royalty', 'Cabin_0', 'Cabin_1'],
      dtype='object')

In [16]:
df_train.fillna(0,inplace=True)

In [17]:
from sklearn import preprocessing

#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_train[["Fare","SibSp","Parch","Age"]])
#标准化处理
df_train[["Fare","SibSp","Parch","Age"]] = preprocessing.scale(X_temp)

In [18]:
df_train.head()

,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,...,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty,Cabin_0,Cabin_1
1,0,-0.099150,0.431350,-0.474326,-0.494675,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0
2,1,0.812389,0.431350,-0.474326,2.525212,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
3,1,0.128735,-0.475199,-0.474326,-0.462841,1,0,0,0,1,...,0,1,0,1,0,0,0,0,1,0
4,1,0.641476,0.431350,-0.474326,0.415536,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
5,0,0.641476,-0.475199,-0.474326,-0.456946,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0


In [19]:
Y = df_train.Survived
X = df_train.drop('Survived',1,inplace=False)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

# LR 模型训练

In [21]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train, y_train)

print(u'LR模型的平均正确率为：%s' % lr.score(x_test, y_test))

LR模型的平均正确率为：0.8239700374531835


# 预测

In [22]:
df_test = pd.read_csv('data/test.csv',header=None)
df_test.columns = ['PassengerId','Pclass','Name','Sex','Age',"SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
df_test.drop(index=0,inplace=True)

In [23]:
df_temp = df_test.copy()

In [24]:
df_test.drop("PassengerId",1,inplace=True)

df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
df_test['Title'] = df_test['Title'].map(title_Dict)
df_test.drop("Name",1,inplace=True)


df_test.loc[df_test.Cabin.isnull(), 'Cabin'] = 'U0'
df_test['Cabin'] = df_test['Cabin'].apply(lambda x: 0 if x == 'U0' else 1)

df_test['Fare'] = df_test['Fare'].astype("float")
df_test['Fare'] = df_test[['Fare']].fillna(df_test.groupby('Pclass').transform(np.mean))

#船票编号特征处理
df_test['Fare'] = df_test['Fare'].astype("float")
df_test['Group_Ticket'] = df_test['Fare'].groupby(by=df_test['Ticket']).transform('count')
df_test['Fare'] = df_test['Fare'] / df_test['Group_Ticket']
df_test.drop(['Group_Ticket'], axis=1, inplace=True)
df_test.drop("Ticket",1,inplace=True)

df_test = pd.get_dummies(df_test,columns = ["Sex","Pclass","Embarked","Title",'Cabin'])
df_test.fillna(0,inplace=True)

print(df_test.columns)

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty', 'Cabin_0', 'Cabin_1'],
      dtype='object')


In [25]:
from sklearn import preprocessing

#归一化处理
min_max_scaler = preprocessing.MinMaxScaler()
X_temp = min_max_scaler.fit_transform(df_test[["Fare","SibSp","Parch","Age"]])

#标准化处理
df_test[["Fare","SibSp","Parch","Age"]] = preprocessing.scale(X_temp)

In [26]:
Predict = lr.predict(df_test)

In [27]:
df_test.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Pclass_1',
       'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',
       'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer',
       'Title_Royalty', 'Cabin_0', 'Cabin_1'],
      dtype='object')

In [28]:
# submission
df = pd.DataFrame({"PassengerId": df_temp["PassengerId"].values, "Survived": Predict})
df.to_csv("gender_submission.csv", index=False)